In [1]:
import requests
import json
import xmltodict
import urllib.parse
import opencitingpy
import datetime
import sqlite3
import os
from matplotlib import pyplot as plt
import numpy as np
import re
import math
import ipywidgets as widgets
from IPython.display import display, clear_output
import re 

# Setup sqlite Datenbank
Datenbank wird im Verzeichnis "C:\MA_Pethke_3992454" abgelegt

In [2]:
# Anlegen von Datenbankspeicherort, falls er nicht existiert
path = "C:\MA_Pethke_3992454"
if not os.path.exists(path):
    os.makedirs(path)

# Verbindung zu DB aufbauen, DB wird automatisch erstellt, wenn keine da ist
con = sqlite3.connect("C:\MA_Pethke_3992454\MA_3992454_idea_1.db")
cur = con.cursor()

In [3]:
## Tabellen Schema erstellen
#------------------- Tabelle Dokument --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Dokumente(PMID INTEGER UNIQUE, titel TEXT, pubdate DATE)")

#------------------- Tabelle Zitationen --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Zitationen(PMID INTEGER, CitingPMID INETEGER)")

#------------------- Tabelle Mesh --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Mesh(UI VARCHAR UNIQUE, name TEXT, datum DATE)")

#------------------- Tabelle Mesh - PMID --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Mesh_PMID(UI VARCHAR, PMID INTEGER)")

#------------------- Tabelle Abfrage --------------------------------------#
cur.execute("CREATE TABLE IF NOT EXISTS Abfrage(QID INTEGER PRIMARY KEY AUTOINCREMENT, query TEXT, PMID INTEGER, ranking INTEGER, mesh VARCHAR)")
con.commit()

# Allgemeine Abfragen aus DB

In [4]:
def getPubDate(pmid):
    cur.execute("SELECT strftime('%Y', pubdate) FROM Dokumente WHERE PMID = \""+ str(pmid)+ "\"")
    pubdate = int(cur.fetchall()[0][0])
    return pubdate

In [79]:
def getMeSHYear(meshUI):
    cur.execute("SELECT strftime('%Y', datum) FROM Mesh WHERE UI = \""+ str(meshUI)+ "\"")
    meshYear = int(cur.fetchall()[0][0])
    return meshYear

In [6]:
def getAllMesh():
    cur.execute("SELECT name FROM Mesh")
    meshs = [x[0] for x in cur.fetchall()]
    return meshs


In [28]:
def getPMIDsToMesh_withCitations(meshUI_loc):
    cur.execute("SELECT distinct Mesh_PMID.PMID FROM Mesh_PMID JOIN Zitationen ON Mesh_PMID.PMID = Zitationen.PMID where Mesh_PMID.UI = '" + str(meshUI_loc) + "'")
    # cur.execute("SELECT PMID from Mesh_PMID where UI = \"" + str(meshUI_loc) + "\"")
    pmids = [x[0] for x in cur.fetchall()]
    return pmids

def getPMIDsToMesh_withOUTCitations(meshUI_loc):
    cur.execute("SELECT distinct PMID FROM Mesh_PMID where PMID not in (SELECT distinct PMID FROM Zitationen) and UI = '" + str(meshUI_loc) + "'")
    pmids = [x[0] for x in cur.fetchall()]
    return pmids

In [73]:
def getMeshUIWithMeshName(mesh_name):
    cur.execute("SELECT UI from Mesh where name = \"" + str(mesh_name) + "\"")
    mUI = [x[0] for x in cur.fetchall()]
    return mUI[0]


# Allgemeine Inserts in DB

In [9]:
def insertMeshData_db(meshUI_loc, meshTerm_loc, meshDate_loc):
    print("Das ausgewählte MeSH " + str(meshUI_loc[0]) + " wird in die DB eingepflegt")
    try:
        cur.execute("INSERT INTO Mesh (UI, name, datum) VALUES (\""+str(meshUI_loc[0])+"\", \""+str(meshTerm_loc)+"\", \""+str(meshDate_loc)+"\")")
        con.commit()
        # print("Insert erfolgreich")
    except:
        return
        # print("MeSH schon in DB")

In [10]:
global resource_auswahl
global label_auswahl

# Dokumente zu MeSH von PubMed

In [19]:
def insertMeshPmid(meshUI_loc, pmidList):
    print("Die 10 'relevantesten' Dokumente, welche das ausgwählte MeSH zugeordnet haben werden in die DB eingepflegt.")
    for pmid in pmidList:
        try:
            cur.execute('INSERT INTO Mesh_PMID (UI, PMID) VALUES ("'+str(meshUI_loc)+'", "'+str(pmid)+'")')
            con.commit()
        except:
            continue


def getDokDataFromPmid(pmid):
    # Daten zu einzelner PMID
    requ = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id="+urllib.parse.quote(str(pmid)))
    # Formatierung
    xpars = xmltodict.parse(requ.text)
    json_str = json.dumps(xpars, indent=4)
    json_data = json.loads(json_str)

    ## Titel 
    titleWithout = ""
    try:
        title = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"]["ArticleTitle"]
        titleWithout = title.replace('"', "'")
    except:
        try:
            title = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"]["ArticleTitle"]["#text"]
            titleWithout = title.replace('"', "'")
        except:
            print("Kein Titel zu PMID:", str(pmid))

    ## PubDate
    pubDate = datetime.datetime(1,1,1)
    # Nicht alle Dokumente haben ein eigenes Veröffentlichungsdatum, dann wird sich auf das DateCompleted von PubMed berufen (TODO: recherchieren, was das genau ist)
    try:
        pubDateAll = json_data["PubmedArticleSet"]["PubmedArticle"]["MedlineCitation"]["Article"]["ArticleDate"]
        pubDate = datetime.datetime(int(pubDateAll["Year"]), int(pubDateAll["Month"]), int(pubDateAll["Day"]))
    except:
        pubDateAll = json_data["PubmedArticleSet"]["PubmedArticle"]["PubmedData"]["History"]["PubMedPubDate"][0]
        pubDate = datetime.datetime(int(pubDateAll["Year"]), int(pubDateAll["Month"]), int(pubDateAll["Day"]))
    return titleWithout, pubDate

def insertDokData_db(pmidList_loc):
    print("Die zu den Dokumenten gehörenden Daten werden in die DB eingepflegt.")
    for pmid in pmidList_loc:
        # print(pmid)
        try:
            titleWithout, pubDate = getDokDataFromPmid(pmid)
        except:
            titleWithout, pubDate = "ERROR", datetime.datetime(1,1,1)
        
        try:
            cur.execute('INSERT INTO Dokumente (PMID, titel, pubdate) VALUES ("'+str(pmid)+'", "'+str(titleWithout)+'", "'+str(pubDate)+'")')
            con.commit()
        except:
            continue
            # print("Dokument schon in DB")


In [12]:
def getPMIDsWithMeSH(meshUI, meshTerm):
    limit = 20 # API limited to first 10000 documents; esearch is a way to get all data, but u have to work on an unix environment https://dataguide.nlm.nih.gov/edirect/edirect-vs-e-utilities.html
    # Abfrage von Dokumente, die vor der Aufnahme des MeSHs veröffentlicht wurden (Von 1900 an)
    meshYear = getMeSHYear(meshUI)
    requ = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=("+urllib.parse.quote(meshTerm)+"[MeSH+Terms])+AND+((\"1900\"[Date+-+Publication]+%3A+\""+str(meshYear)+"\"[Date+-+Publication]))&retmax="+str(limit)+"&sort=relevance&retmode=json&offset=")
    requ_json = json.loads(requ.text)
    global pmidList
    pmidList = requ_json['esearchresult']['idlist']
    global countResult
    countResult = requ_json['esearchresult']['count']

# MeSH Data

In [13]:
def saveDropDownChoose(MeSH):
    global drdMesh
    drdMesh = MeSH

In [14]:
def chooseMeSH(MeSH):
    index = label_auswahl.index(MeSH)
    resource = resource_auswahl[index]
    global meshUI
    meshUI = re.findall("^.*mesh\/(.*)$", resource)
    requ = requests.get(str(resource)+".json", headers={"Accept": "application/json"})
    meshData = requ.json()
    meshDate = meshData["dateCreated"]
    insertMeshData_db(meshUI, MeSH, meshDate)
    meshUI = meshUI[0]
    meshTerm = MeSH
    getPMIDsWithMeSH(meshUI, meshTerm)

# Zitationsdaten erheben

In [15]:
def getAllCitingPMIDs(pmid):
    response_PubMedMCitations = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&linkname=pubmed_pubmed_citedin&id="+urllib.parse.quote(str(pmid)))
    # Formatierung
    xpars = xmltodict.parse(response_PubMedMCitations.text)
    json_str = json.dumps(xpars, indent=4)
    json_data = json.loads(json_str)
    try:
        citingPMIDs = [x["Id"] for x in json_data["eLinkResult"]["LinkSet"]["LinkSetDb"]["Link"]]
        print("PMID: ", pmid, "Anzahl Zitationen: ", len(citingPMIDs))
    except:
        citingPMIDs = []
        print("PMID: ", pmid, "Anzahl Zitationen: 0")
    return citingPMIDs

def insertCitationPMIDCombi(pmid, citingPmids):
    # Wenn Zitationen eines Dokuments schon erhoben wurden, dann nicht nochmal alle durchgehen
    cur.execute("SELECT count(PMID) FROM Zitationen WHERE PMID = "+ str(pmid))
    anzahlEingetrageneZitationsBeziehungen = cur.fetchall()[0][0]
    if anzahlEingetrageneZitationsBeziehungen != 0:
        return
    else:
        for citingPMID in citingPmids:
            try:
                cur.execute('INSERT INTO Zitationen (PMID, CitingPMID) VALUES ("'+str(pmid)+'", "'+str(citingPMID)+'")')
                con.commit()
            except:
                print("Zitationskombi schon in DB")

def insertCitingPMIDs(pmidList_loc):
    for pmid in pmidList_loc:
        citingPMIDs = getAllCitingPMIDs(pmid)
        # Fügt alle Dokumente in Tabelle "Dokumente" 
        insertDokData_db(citingPMIDs)
        print("Die PMID und zitierende PMIDs werden in die DB eingepflegt")
        insertCitationPMIDCombi(pmid, citingPMIDs)

# Zitationsdaten in Verhältnis zu MeSH

In [16]:
def getDateOfMeSH(meshUI):
    cur.execute("SELECT datum FROM Mesh WHERE UI = \""+ str(meshUI)+ "\"")
    meshDate = datetime.datetime.strptime(cur.fetchall()[0][0], "%Y-%m-%d").date()
    return meshDate


def getCitationDataForPMID(pmid):
    # für jede PMID in idList die zitierenden PMIDs abfragen
    cur.execute("SELECT CitingPMID FROM Zitationen WHERE PMID = \""+ str(pmid)+ "\"")
    citingPMIDs =  [x[0] for x in cur.fetchall()]
    if len(citingPMIDs) == 0:
        # print(citationData)
        # print(pmid, "Zu diesem Dokument liegen keine Zitationsdaten vor")
        citationData = [[], []]
    else:
        # SQLite Abfrage erstellen
        queryCitingPMIDs = "PMID = " + str(citingPMIDs[0])
        if len(citingPMIDs) > 1:
            for citingPMID in citingPMIDs[1:]:
                queryCitingPMIDs = queryCitingPMIDs + " or PMID = " + str(citingPMID)
        query = "SELECT strftime('%Y', pubdate) as year, count(strftime('%Y', pubdate)) as count FROM Dokumente WHERE "+queryCitingPMIDs+" GROUP BY year"
        cur.execute(query)
        output = cur.fetchall()
        years = [int(x[0]) for x in output]
        citationCounts = [int(x[1]) for x in output]
        citationData = [years, citationCounts]
    return citationData

# GUI

## Datenerhebung

In [ ]:
output = widgets.Output()

beschreibung = widgets.HTML(value="Bitte geben Sie ein Stichwort ein")
input_text = widgets.Text(placeholder='Bitte geben Sie ein Stichwort ein')
button_ok_mesh = widgets.Button(description="OK")

display(beschreibung, input_text, button_ok_mesh)



def on_button_clicked_1(b):
    requ = requests.get("https://id.nlm.nih.gov/mesh/lookup/descriptor?label="+str(input_text.value)+"&match=contains&year=current&limit=10", headers={"Accept": "application/json"})
    meshData = requ.json()
    global label_auswahl
    label_auswahl = [output['label'] for output in meshData]
    global resource_auswahl
    resource_auswahl = [output['resource'] for output in meshData]
    # display(output)
    clear_output()
    beschreibung = widgets.HTML(value="Wählen Sie ein existierendes MeSH aus!")
    display(beschreibung)
    drd = widgets.interact(saveDropDownChoose, MeSH=label_auswahl, layout={'width':'max-content'})
    button_ok_pubmed = widgets.Button(description="Start")
    display(drd, button_ok_pubmed)
    button_ok_pubmed.on_click(on_button_clicked_2)

def on_button_clicked_2(b):
    chooseMeSH(drdMesh)
    insertMeshPmid(meshUI, pmidList)
    insertDokData_db(pmidList)
    insertCitingPMIDs(pmidList)
    

button_ok_mesh.on_click(on_button_clicked_1)


## Ergebnisvisualisierung

In [75]:
def saveDropDownChoose_visuell_1(meshDRD):
    global meshUI_glob_1
    meshUI_glob_1 = getMeshUIWithMeshName(meshDRD)

def chooseMeSH_drd(meshName_loc):
    pmids = getPMIDsToMesh_withCitations(meshUI_glob_1)
    pmids_withoutCitations = getPMIDsToMesh_withOUTCitations(meshUI_glob_1)
    print("Für die PMIDs " + str(pmids_withoutCitations) + " liegen keine Zitationsdaten vor.")
    drd_pmid = widgets.interact(fillPlot, pmid=pmids, layout={'width':'max-content'})
    display(drd_pmid)

def fillPlot(pmid):
    clear_output
    # PLOT
    fig, ax = plt.subplots(layout="constrained")
    ax.set_ylabel("Anzahl Zitationen")
    ax.set_xlabel("Jahre")
    pubDate = getPubDate(pmid)
    meshDate = getMeSHYear(meshUI_glob_1)
    years, counts = getCitationDataForPMID(pmid)
    ax.set_title("PMID: " + str(pmid))
    ax.set_xlim(min(pubDate, meshDate)-5, 2023)
    ax.set_ylim(0, 50)
    ax.bar(years, counts)
    ax.annotate("Veröffentlichungsjahr",
        xy=(pubDate, 20), 
        xycoords="data",
        xytext=(0, 50),
        textcoords="offset points", 
        arrowprops=dict(facecolor="blue"))
    ax.annotate("MeSH Aufnahme",
        xy=(meshDate, 20), 
        xycoords="data",
        xytext=(0, 50),
        textcoords="offset points", 
        arrowprops=dict(facecolor="red"))
    ax.plot()

def on_button_clicked_3(b):
    print("MeSH UI: ", meshUI_glob_1)
    chooseMeSH_drd(meshUI_glob_1)

allMeshinDB = getAllMesh()
drd_mesh = widgets.interact(saveDropDownChoose_visuell_1, meshDRD=allMeshinDB, layout={'width':'max-content'})
button_ok_visuell = widgets.Button(description="OK")

display(drd_mesh, button_ok_visuell)    
button_ok_visuell.on_click(on_button_clicked_3)

interactive(children=(Dropdown(description='meshDRD', options=('Aspirin, Dipyridamole Drug Combination', 'Aspi…

<function __main__.saveDropDownChoose_visuell_1(meshDRD)>

Button(description='OK', style=ButtonStyle())

MeSH UI:  D000068342
Für die PMIDs [19951767, 11975816, 17297537, 24161074, 23532686, 18217142, 14680441, 20060783, 20978715, 21415156, 24026956, 16794200, 21502757] liegen keine Zitationsdaten vor.


interactive(children=(Dropdown(description='pmid', options=(11098344, 20410547, 12885264, 19361598, 18343263, …

<function __main__.fillPlot(pmid)>

In [86]:
# Vergleich: Mehr in emergenten oder nicht-emergentem Zeitraum
# abhängig von MeSH UND PMID

def saveDropDownChoose_visuell_2(meshDRD):
    global meshUI_glob_2
    meshUI_glob_2 = getMeshUIWithMeshName(meshDRD)

def getCitationCountBetween(begin, end, choosenPMID):
    cur.execute(" SELECT count(distinct CitingPMID) FROM Zitationen JOIN Dokumente WHERE Zitationen.PMID = "+str(choosenPMID)+" and Zitationen.CitingPMID = Dokumente.PMID and strftime('%Y', Dokumente.pubdate) <= '"+str(end)+ "' and strftime('%Y', Dokumente.pubdate) >= '"+str(begin)+ "'")
    citationCount = cur.fetchall()[0][0]
    return citationCount

def on_button_clicked_4(b):
    meshDate = getMeSHYear(meshUI_glob_2)
    pmids_cited = getPMIDsToMesh_withCitations(meshUI_glob_2)
    choosenPMID = pmids_cited[0]
    pmid_pubdate = getPubDate(choosenPMID)
    zeitraum = meshDate - pmid_pubdate
    jahr_korrektur = min(meshDate + zeitraum, datetime.datetime.today().year) # es kann maximal das aktuelle Jahr angegeben werden

    # Zeitraum Emergenz: Veröffentlichungsjahr - MeSH Aufnahme
    count1 = getCitationCountBetween(pmid_pubdate, meshDate, choosenPMID)
    # Zeitraum Nicht-Emergenz: MeSH Aufnahme - jetzt
    count2 = getCitationCountBetween(meshDate, int(datetime.datetime.today().year), choosenPMID)
    # Zeitraum Nicht-Emergenz angepasst: MeSH Aufnahme + Zeitraum Emergenz
    count3 = getCitationCountBetween(meshDate, jahr_korrektur, choosenPMID)

    print("Anzahl Zitationen vor MeSH-Aufnahme (", str(pmid_pubdate), " - ", str(meshDate), ") für PMID: ", str(choosenPMID), ", EMERGENT:", count1)
    print("Anzahl Zitationen vor MeSH-Aufnahme (", str(meshDate), " - ", str(datetime.datetime.today().year), ") für PMID: ", str(choosenPMID), ", NICHT-emergent:", count2)
    print("Anzahl Zitationen vor MeSH-Aufnahme (", str(meshDate), " - ", str(jahr_korrektur), ") für PMID: ", str(choosenPMID), ", nicht-emergent KORRIGIERT:", count3)



allMeshinDB = getAllMesh()
drd_mesh = widgets.interact(saveDropDownChoose_visuell_2, meshDRD=allMeshinDB, layout={'width':'max-content'})
button_ok_visuell = widgets.Button(description="OK")

display(drd_mesh, button_ok_visuell)    
button_ok_visuell.on_click(on_button_clicked_4)

interactive(children=(Dropdown(description='meshDRD', options=('Aspirin, Dipyridamole Drug Combination', 'Aspi…

<function __main__.saveDropDownChoose_visuell_2(meshDRD)>

Button(description='OK', style=ButtonStyle())

Anzahl Zitationen vor MeSH-Aufnahme ( 2000  -  2015 ) für PMID:  11098344 , EMERGENT: 8
Anzahl Zitationen vor MeSH-Aufnahme ( 2015  -  2023 ) für PMID:  11098344 , NICHT-emergent: 5
Anzahl Zitationen vor MeSH-Aufnahme ( 2015  -  2023 ) für PMID:  11098344 , nicht-emergent KORRIGIERT: 5


In [90]:
# Vergleich: Mehr in emergenten oder nicht-emergentem Zeitraum
# abhängig von MeSH 

def saveDropDownChoose_visuell_3(meshDRD):
    global meshUI_glob_3
    meshUI_glob_3 = getMeshUIWithMeshName(meshDRD)

def getCitationCountBetween_Mesh(begin, end, meshUI_loc):
    cur.execute(" SELECT count(distinct CitingPMID) FROM Zitationen JOIN Dokumente WHERE Zitationen.PMID in (SELECT distinct PMID FROM Mesh_PMID WHERE UI = '"+str(meshUI_loc)+"') and Zitationen.CitingPMID = Dokumente.PMID and strftime('%Y', Dokumente.pubdate) <= '"+str(end)+ "' and strftime('%Y', Dokumente.pubdate) >= '"+str(begin)+ "'")
    citationCount = cur.fetchall()[0][0]

    return citationCount

def getearliestDokPubDate(meshUI_loc):
    cur.execute(" SELECT min(strftime('%Y', Dokumente.pubdate)) FROM Zitationen JOIN Dokumente WHERE Zitationen.PMID in (SELECT distinct PMID FROM Mesh_PMID WHERE UI = '"+str(meshUI_loc)+"') and Zitationen.CitingPMID = Dokumente.PMID")
    earliestPubDate = cur.fetchall()[0][0]
    return earliestPubDate

def on_button_clicked_5(b):
    print("MeSH UI: ", meshUI_glob_3)
    meshDate = getMeSHYear(meshUI_glob_3)
    earliestDokPubDate = getearliestDokPubDate(meshUI_glob_3)
    zeitraum = meshDate - int(earliestDokPubDate)
    jahr_korrektur = min(meshDate + zeitraum, datetime.datetime.today().year) # es kann maximal das aktuelle Jahr angegeben werden

    # Zeitraum Emergenz: Veröffentlichungsjahr - MeSH Aufnahme
    count1 = getCitationCountBetween_Mesh(earliestDokPubDate, meshDate, meshUI_glob_3)
    # Zeitraum Nicht-Emergenz: MeSH Aufnahme - jetzt
    count2 = getCitationCountBetween_Mesh(meshDate, int(datetime.datetime.today().year), meshUI_glob_3)
    # Zeitraum Nicht-Emergenz angepasst: MeSH Aufnahme + Zeitraum Emergenz
    count3 = getCitationCountBetween_Mesh(meshDate, jahr_korrektur, meshUI_glob_3)


    print("Anzahl Zitationen vor MeSH-Aufnahme (", str(earliestDokPubDate), " - ", str(meshDate), "), EMERGENT:", count1)
    print("Anzahl Zitationen vor MeSH-Aufnahme (", str(meshDate), " - ", str(datetime.datetime.today().year), "), NICHT-emergent:", count2)
    print("Anzahl Zitationen vor MeSH-Aufnahme (", str(meshDate), " - ", str(jahr_korrektur), "), nicht-emergent KORRIGIERT:", count3)


allMeshinDB = getAllMesh()
drd_mesh = widgets.interact(saveDropDownChoose_visuell_3, meshDRD=allMeshinDB, layout={'width':'max-content'})
button_ok_visuell = widgets.Button(description="OK")

display(drd_mesh, button_ok_visuell)    
button_ok_visuell.on_click(on_button_clicked_5)

interactive(children=(Dropdown(description='meshDRD', options=('Aspirin, Dipyridamole Drug Combination', 'Aspi…

<function __main__.saveDropDownChoose_visuell_3(meshDRD)>

Button(description='OK', style=ButtonStyle())